# 1. Introducción

En este cuaderno vamos a realizar la transposición de una matriz cuadrada. Para esto, previamente debemos indicar el valor "m" en los parámetros de entrada, el cual nos indicará las dimensiones de la matriz a procesar.


Primeramente se convertirá la matriz en un array unidimensional y luego se la pasará por parámetros a la función kernel_vec, en está función se irá recorriendo dicho vector, asignando en un vector de salida la transposición realizada.

# 2. Armado del ambiente

## 2.1 Módulo CUDA

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 66 kB 3.7 MB/s 
     |████████████████████████████████| 75 kB 3.8 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627577 sha256=bad209aa7a5de54af41b8318f43d35dffb5906ed631c7833a375f8861b8e7a33
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.9-py2.py3-none-any.whl size=62370 sha256=3e5a7c30f580f74a18095064140b92232328b8d96be9aab8d2e009417fae1f42
  Stored in directory: /root/.cache/pip/wheels/41/b9/6e/94bb014f6484b15ec77e7877f3a227609481ffd98db364504d
Successfully built pycuda pytools


#3. Desarrollo CPU

In [ ]:
%matplotlib inline
from datetime import datetime
tiempo_total = datetime.now()

import matplotlib.pyplot as plt
import numpy

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# --------------------------------------------

m = 1000#@param {type: "number"}

tiempo_memoria = datetime.now()

matriz = numpy.zeros((m,m)).astype(numpy.int32())
matriz_final=numpy.zeros((m,m)).astype(numpy.int32())

tiempo_memoria = datetime.now() - tiempo_memoria

############ CREACION DE MATRIZ #####################
tiempo_mat_carga = datetime.now()
k=1
for i in range(0, m):
  for j in range(0, m):
    matriz[i][j]=k
    k=k+1
tiempo_mat_carga = datetime.now() - tiempo_mat_carga
############ FIN CREACION DE MATRIZ #################

############ TRANSPOSICION DE MATRIZ ################
#cuento el tiempo
tiempo_mat = datetime.now()

for i in range(0, m):
  for j in range(0, m):
    matriz_final[j][i]=matriz[i][j]

#obtengo el tiempo que tardo
tiempo_mat = datetime.now() - tiempo_mat
############ FIN TRANSPOSICION DE MATRIZ ############

#-----------------------------------------------------------

tiempo_total = datetime.now() - tiempo_total
print("Matriz original:")
print(matriz)

print("\n\nMatriz transpuesta:")
print(matriz_final) 

print( "\nTiempo de creación de matriz: ", tiempo_en_ms( tiempo_mat_carga), "[ms]")
print( "Tiempo de transposición: ", tiempo_en_ms( tiempo_mat), "[ms]")
print( "Tiempo en reservar memoria: ", tiempo_en_ms( tiempo_memoria ), "[ms]")
print( "Tiempo Total:", tiempo_en_ms( tiempo_total ), "[ms]" )

Matriz original:
[[      1       2       3 ...     998     999    1000]
 [   1001    1002    1003 ...    1998    1999    2000]
 [   2001    2002    2003 ...    2998    2999    3000]
 ...
 [ 997001  997002  997003 ...  997998  997999  998000]
 [ 998001  998002  998003 ...  998998  998999  999000]
 [ 999001  999002  999003 ...  999998  999999 1000000]]


Matriz transpuesta:
[[      1    1001    2001 ...  997001  998001  999001]
 [      2    1002    2002 ...  997002  998002  999002]
 [      3    1003    2003 ...  997003  998003  999003]
 ...
 [    998    1998    2998 ...  997998  998998  999998]
 [    999    1999    2999 ...  997999  998999  999999]
 [   1000    2000    3000 ...  998000  999000 1000000]]

Tiempo de creación de matriz:  434.52 [ms]
Tiempo de transposición:  746.542 [ms]
Tiempo en reservar memoria:  7.153 [ms]
Tiempo Total: 1188.451 [ms]


#4. Desarrollo GPU

In [ ]:
%matplotlib inline
from datetime import datetime
tiempo_total = datetime.now()

import numpy
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# --------------------------------------------

m = 1000#@param {type: "number"}

tiempo_memoria = datetime.now()

#defino el vector que representa la matriz
vector = numpy.zeros(m*m).astype(numpy.int32())
#defino el vector que representa a la matriz transpuesta
vector_final = numpy.zeros(m*m).astype(numpy.int32())

# Reservo los 2 arrays en GPU
matriz_gpu       = cuda.mem_alloc( vector.nbytes)
matriz_final_gpu = cuda.mem_alloc( vector.nbytes)

# GPU - Copio la memoria al GPU
#cuda.memcpy_htod( matriz_gpu, vector )

tiempo_memoria = datetime.now() - tiempo_memoria

#CPU - Defino la función kernel de carga de la matriz que ejecutará en GPU
module_def_matriz = SourceModule("""
__global__ void kernel_vec_carga( int m, int *matriz )
{
  // Calculo las coordenadas del Thread en dos dimensiones.
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  int idy = threadIdx.y + blockIdx.y*blockDim.y;

  if(idx<m && idy<m)
    matriz[idx+idy*m]=idx+idy*m;
}
""")
#CPU - Defino la función kernel de transposicion de la matriz que ejecutará en GPU
module = SourceModule("""
__global__ void kernel_vec_transposicion( int m, int *matriz, int *matrizRes )
{
  // Calculo las coordenadas del Thread en dos dimensiones.
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  int idy = threadIdx.y + blockIdx.y*blockDim.y;

  if(idx<m && idy <m)
    matrizRes[idx*m+idy]=matriz[idy*m+idx];
}
""")

dim_hilo_x = 16
dim_bloque_x = numpy.int( (m+dim_hilo_x-1) / dim_hilo_x )

dim_hilo_y = 16
dim_bloque_y = numpy.int( (m+dim_hilo_y-1) / dim_hilo_y )

##################### CARGA MATRIZ ########################
#cuento el tiempo 
tiempo_mat_carga = datetime.now()

#CPU - Genero la función kernel.
kernel = module_def_matriz.get_function("kernel_vec_carga")

#LLamo a la función kernel
kernel( numpy.int32(m), matriz_gpu, block=( dim_hilo_x, dim_hilo_y, 1 ), grid=(dim_bloque_x, dim_bloque_y,1) )

#obtengo el tiempo que tardo en cargar la matriz
tiempo_mat_carga = datetime.now() - tiempo_mat_carga
##################### FIN CARGA MATRIZ #####################

##################### TRANSPOSICION MATRIZ #################
#cuento el tiempo 
tiempo_mat = datetime.now()

#CPU - Genero la función kernel.
kernel = module.get_function("kernel_vec_transposicion")

#LLamo a la función kernel
kernel( numpy.int32(m), matriz_gpu, matriz_final_gpu, block=( dim_hilo_x, dim_hilo_y, 1 ), grid=(dim_bloque_x, dim_bloque_y,1) )

#obtengo el tiempo que tardo en cargar la matriz
tiempo_mat = datetime.now() - tiempo_mat
##################### FIN TRANSPOSICION MATRIZ #############

# GPU - copio la matriz original desde la memoria GPU
cuda.memcpy_dtoh( vector, matriz_gpu )
matriz = numpy.reshape(vector,(m,m))

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( vector_final, matriz_final_gpu )
matriz_final = numpy.reshape(vector_final,(m,m))

#-----------------------------------------------------------

tiempo_total = datetime.now() - tiempo_total

print( "Thread: [", dim_hilo_x, ",", dim_hilo_y, " ], Bloque : [", dim_bloque_x, ",", dim_bloque_y, "]" )
print( "Total de Thread: [", dim_hilo_x*dim_bloque_x, ",", dim_hilo_y*dim_bloque_y, " ]", " = ", dim_hilo_x*dim_bloque_x*dim_hilo_y*dim_bloque_y )

print("Matriz original:")
print(matriz)

print("\n\nMatriz transpuesta:")
print(matriz_final) 

print( "\nTiempo de creación de matriz GPU ", tiempo_en_ms( tiempo_mat_carga ), "[ms]" )
print( "Tiempo de transposición de matriz GPU  : ", tiempo_en_ms( tiempo_mat ), "[ms]" )
print( "Tiempo en reservar memoria en CPU-GPU: ", tiempo_en_ms( tiempo_memoria ), "[ms]")
print( "Tiempo Total GPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )

Thread: [ 16 , 16  ], Bloque : [ 63 , 63 ]
Total de Thread: [ 1008 , 1008  ]  =  1016064
Matriz original:
[[     0      1      2 ...    997    998    999]
 [  1000   1001   1002 ...   1997   1998   1999]
 [  2000   2001   2002 ...   2997   2998   2999]
 ...
 [997000 997001 997002 ... 997997 997998 997999]
 [998000 998001 998002 ... 998997 998998 998999]
 [999000 999001 999002 ... 999997 999998 999999]]


Matriz transpuesta:
[[     0   1000   2000 ... 997000 998000 999000]
 [     1   1001   2001 ... 997001 998001 999001]
 [     2   1002   2002 ... 997002 998002 999002]
 ...
 [   997   1997   2997 ... 997997 998997 999997]
 [   998   1998   2998 ... 997998 998998 999998]
 [   999   1999   2999 ... 997999 998999 999999]]

Tiempo de creación de matriz GPU  0.18 [ms]
Tiempo de transposición de matriz GPU  :  0.159 [ms]
Tiempo en reservar memoria en CPU-GPU:  5.506 [ms]
Tiempo Total GPU:  9.538 [ms]


#5. Métricas



**Métricas obtenidas de las velocidades de procesamiento entre CPU y GPU**

**CPU**

Tiempo de creación de la matriz: 432.52 [ms]

Tiempo de procesamiento (transposición de matriz): 746.542 [ms]

Tiempo total: 1188.451 [ms]

**GPU**

Tiempo de creación de la matriz: 0.18 [ms]

Tiempo de procesamiento (transposición de matriz): 0.159 [ms]

Tiempo total: 9.538 [ms]

#6. Conclusión

Como conclusión podemos observar que la velocidad de procesamiento de GPU tiene una diferencia enorme respecto a la de CPU al realizar el intercambio de posiciones cuando el valor de dimensión de la matriz es muy grande.

#7. Bibliografía

*   https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb
*  https://www.cs.colostate.edu/~cs675/MatrixTranspose.pdf
*  https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.232.4776&rep=rep1&type=pdf
